### Before start, please replace every folder path

In [1]:
import pandas as pd
import numpy as np
import copy
from math import *
import os
import random
from collections import defaultdict
import csv
from functools import reduce
from copy import deepcopy
import pickle
from timeit import default_timer as timer
from matplotlib import pyplot as plt
from ase import Atoms
from ase.io import read
from ase.io import write
from ase.visualize import view
import itertools

%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 500)

In [5]:
folder_path = 'CHA'

In [6]:
%run helper_functions.py

#### Read lat.in

In [5]:
filepath=folder_path+'/lat.in'
lattice = read_lat_in(filepath)

In [6]:
lattice.keys()

dict_keys(['a', 'b', 'c', 'alpha', 'beta', 'gamma', 'u', 'v', 'w', 'sites'])

#### Run corrdump, generate clusters.out

In [11]:
#need to change path before running the following command
!corrdump -l=CHA/lat.in -cf=CHA/clusters.out -2=7 -3=7

Unable to open structure file


#### Read clusters.out

In [12]:
filepath = folder_path+'/clusters.out'
cluster_out_lines = read_clusters_out(filepath)

In [13]:
lat_clusters = break_down_cluster_out_lines(cluster_out_lines, lattice)

In [14]:
lat_clusters.keys()

dict_keys(['type_number', '0-1', 'num_max_site', '1-1', '2-1', '2-2', '2-3', '2-4', '2-5', '2-6', '2-7', '2-8', '2-9', '2-10', '2-11', '2-12', '2-13', '2-14', '3-1', '3-2', '3-3', '3-4', '3-5', '3-6', '3-7', '3-8', '3-9', '3-10', '3-11', '3-12', '3-13', '3-14', '3-15', '3-16', '3-17', '3-18', '3-19', '3-20', '3-21', '3-22', '3-23', '3-24', '3-25', '3-26', '3-27', '3-28', '3-29', '3-30', 'vis_range', 'lat_vis_sites', 'cluster_types'])

In [15]:
#need to change path before running the following command
!mkdir CHA/lattice_clusters
!mkdir CHA/lattice_clusters/xyzs
!mkdir CHA/lattice_clusters/images

In [16]:
str_vec = [0 for i in range(len(lat_clusters['lat_vis_sites'].index))]

In [17]:
visualize_str_no_rep(lat_clusters['lat_vis_sites'], str_vec, folder_path+'/lattice_clusters/xyzs/all_sites.xyz', folder_path+'/lattice_clusters/images/all_sites.png')
#visualize_str_no_rep(lat_clusters['lat_vis_sites'], str_vec, folder_path+'/lattice_clusters/xyzs/all_sites.xyz', folder_path+'/lattice_clusters/images/all_sites.png')
#c= read(folder_path+'/lattice_clusters/xyzs/all_sites.xyz')
#view(c)

In [18]:
#visualize all n-body clusters
nsite = 3
for i in range(1, int(lat_clusters['type_number'][nsite])+1):
    #print(i)
    str_vec = [0 for i in range(len(lat_clusters['lat_vis_sites'].index))]
    sites = lat_clusters['{}-{}'.format(nsite, i)]['eg_pair_frac']
    for site in sites:
        #print(site)
        index = find_df_index_frac(site, lat_clusters['lat_vis_sites'])
        str_vec[index] = 1
    xyz_filepath = folder_path+'/lattice_clusters/xyzs/cluster-{}-{}.xyz'.format(nsite, i)
    png_filepath = folder_path+'/lattice_clusters/images/cluster-{}-{}.png'.format(nsite, i)
    visualize_str_no_rep(lat_clusters['lat_vis_sites'], str_vec, xyz_filepath, png_filepath)

In [58]:
c= read(folder_path+'/lattice_clusters/xyzs/cluster-3-6.xyz')
view(c)

#### read str_dim.txt

In [19]:
%run helper_functions.py

In [20]:
filepath = folder_path+'/str_dim.txt'
structure = read_str_dim(filepath, lattice)

In [21]:
structure.keys()

dict_keys(['lattice_a', 'lattice_b', 'lattice_c', 'lattice_alpha', 'lattice_beta', 'lattice_gamma', 'lattice_u', 'lattice_v', 'lattice_w', 'u', 'v', 'w', 'u_mult', 'v_mult', 'w_mult', 'ncell', 'str_sites'])

In [22]:
filepath = folder_path+'/str.out'
prepare_str_out(structure, filepath)

In [34]:
#need to change path before running the following command
if os.path.isfile('CHA/cluster_list.csv'):
    os.remove('CHA/cluster_list.csv')
    
!corrdump -l=CHA/lat.in -s=CHA/str.out -cf=CHA/clusters.out -2=7 -3=7 >> CHA/cluster_list.csv

#### read cluster list

In [35]:
%run helper_functions.py

In [36]:
filepath = folder_path+'/cluster_list.csv'
str_clusters = read_cluster_list(filepath, structure, lat_clusters)

In [37]:
break_down_cluster_list(str_clusters, structure)

In [38]:
#need to change path before running the following command
!mkdir CHA/structure_clusters_rep
!mkdir CHA/structure_clusters_rep/xyzs
!mkdir CHA/structure_clusters_rep/images

mkdir: CHA/structure_clusters_rep: File exists
mkdir: CHA/structure_clusters_rep/xyzs: File exists
mkdir: CHA/structure_clusters_rep/images: File exists


In [39]:
#need to change path before running the following command
!mkdir CHA/structure_clusters_no_rep
!mkdir CHA/structure_clusters_no_rep/xyzs
!mkdir CHA/structure_clusters_no_rep/images

mkdir: CHA/structure_clusters_no_rep: File exists
mkdir: CHA/structure_clusters_no_rep/xyzs: File exists
mkdir: CHA/structure_clusters_no_rep/images: File exists


In [40]:
#visualize_structure_all_clusters(str_clusters, structure, folder_path, rep='y')

#### pair sites for structure sites

In [41]:
%run helper_functions.py

In [42]:
str_clusters.keys()

dict_keys(['cluster_orig_list', 'type_number', 'cluster_types', '1-1', '2-1', '2-2', '2-3', '2-4', '2-5', '2-6', '2-7', '2-8', '2-9', '2-10', '2-11', '2-12', '2-13', '2-14', '3-1', '3-2', '3-3', '3-4', '3-5', '3-6', '3-7', '3-8', '3-9', '3-10', '3-11', '3-12', '3-13', '3-14', '3-15', '3-16', '3-17', '3-18', '3-19', '3-20', '3-21', '3-22', '3-23', '3-24', '3-25', '3-26', '3-27', '3-28', '3-29', '3-30', 'mult_vis_range', 'str_vis_sites'])

In [43]:
add_cluster_list_site_index(str_clusters,structure)

In [44]:
add_pair_sites(structure, str_clusters)

In [47]:
structure['str_sites'].columns

Index(['a', 'b', 'c', 'atom', 'multi_atoms', 'site_index', 'x', 'y', 'z',
       '1-1', '2-1', '2-2', '2-3', '2-4', '2-5', '2-6', '2-7', '2-8', '2-9',
       '2-10', '2-11', '2-12', '2-13', '2-14', '3-1', '3-2', '3-3', '3-4',
       '3-5', '3-6', '3-7', '3-8', '3-9', '3-10', '3-11', '3-12', '3-13',
       '3-14', '3-15', '3-16', '3-17', '3-18', '3-19', '3-20', '3-21', '3-22',
       '3-23', '3-24', '3-25', '3-26', '3-27', '3-28', '3-29', '3-30'],
      dtype='object')

#### count clusters for a given configuration

In [48]:
%run helper_functions.py

In [49]:
str_vec = [1 for i in range(len(structure['str_sites'].index))]

In [50]:
counting_types = []

In [51]:
count_clusters_str_config(str_vec, structure, str_clusters, counting_types)

defaultdict(int,
            {'1-1': 972,
             '2-1': 486,
             '2-10': 486,
             '2-11': 486,
             '2-12': 486,
             '2-13': 972,
             '2-14': 486,
             '2-2': 486,
             '2-3': 486,
             '2-4': 486,
             '2-5': 972,
             '2-6': 486,
             '2-7': 972,
             '2-8': 972,
             '2-9': 972,
             '3-1': 972,
             '3-10': 972,
             '3-11': 972,
             '3-12': 972,
             '3-13': 972,
             '3-14': 972,
             '3-15': 972,
             '3-16': 972,
             '3-17': 972,
             '3-18': 972,
             '3-19': 972,
             '3-2': 972,
             '3-20': 972,
             '3-21': 972,
             '3-22': 972,
             '3-23': 972,
             '3-24': 972,
             '3-25': 972,
             '3-26': 972,
             '3-27': 972,
             '3-28': 972,
             '3-29': 972,
             '3-3': 972,
        

In [60]:
count_clusters_str_config(str_vec, structure, str_clusters, counting_types=['2-7','2-11'], excluding_types=['3-6'])

defaultdict(int, {'2-11': 486, '2-7': 0})

In [61]:
for i in range(15):
    print('3-{}:'.format(i+1),lat_clusters['3-{}'.format(i+1)]['max_d'])

3-1: 4.38555
3-2: 4.38555
3-3: 4.38694
3-4: 5.36602
3-5: 5.36602
3-6: 5.36602
3-7: 5.53411
3-8: 5.53411
3-9: 5.73597
3-10: 5.73597
3-11: 5.73597
3-12: 6.19605
3-13: 6.19605
3-14: 6.19605
3-15: 6.19614


#### random generate one structure configuration

In [144]:
%run helper_functions.py

In [63]:
#need to change path before running the following command
!mkdir CHA/random_config_process
!mkdir CHA/random_config_process/xyzs
!mkdir CHA/random_config_process/images

In [64]:
penalty = {'2-1':1, '2-2':1, '2-3':1,'2-4':1}

str_vec = [1 for i in range(len(structure['str_sites'].index))]
cal_penalty(str_vec, structure, str_clusters, penalty)

1944

In [141]:
str_vec = random_config_hybrid(structure, str_clusters, 100, penalty, vis=0, folder_path='', max_try = 100)

In [147]:
str_vec = random_config_swap(structure, str_clusters, 100, penalty, vis=0, folder_path='', max_try = 100)

#### titrate clusters for one structure configuration

In [149]:
%run helper_functions.py

In [150]:
cluster_type_dict = prepare_cluster_type_dict(str_clusters)

In [151]:
penalty

{'2-1': 1, '2-2': 1, '2-3': 1, '2-4': 1}

In [159]:
str_vec = random_config_hybrid(structure, str_clusters, 340, penalty, max_try = 100)

In [160]:
np.sum(str_vec)

340

In [161]:
cal_penalty(str_vec,structure, str_clusters, penalty)

0

In [162]:
d1 = titrate_config_one_group(str_vec, structure, str_clusters, cluster_type_dict, counting_types=['2-7','2-11'],excluding_types=['3-6'])

In [163]:
d2 = titrate_config_multi_groups(str_vec, structure, str_clusters, cluster_type_dict, counting_groups=[['2-7','2-11']], excluding_types=['3-6'], repeat_num = 10)

In [164]:
d2['2-7']

[60, 60, 60, 60, 60, 60, 60, 60, 60, 60]

In [165]:
d2['2-11']

[47, 47, 47, 47, 47, 47, 47, 47, 47, 47]

#### generate multiple random structure configurations for one compositions

In [166]:
#need to change path before running the following command
!mkdir CHA/random_configs
!mkdir CHA/random_configs/xyzs
!mkdir CHA/random_configs/images

mkdir: CHA/random_configs: File exists
mkdir: CHA/random_configs/xyzs: File exists
mkdir: CHA/random_configs/images: File exists


In [167]:
%run helper_functions.py

In [186]:
str_vecs = random_config_mult(structure, str_clusters, 250, penalty, 10, vis=1, folder_path=folder_path, max_try = 100)

In [ ]:
c= read(folder_path+'/random_configs/images/config-9.xyz')
view(c)

## CHA vs FER

#### count for all Al

In [269]:
%run helper_functions.py

In [46]:
values1 = [1.5]
values2 = [1]
errors1 = [0]
errors2 = [0]

In [ ]:
plot_two_bar_charts('CHA vs FER.png', ['2NN and 3NN Al-Al in 6 membered-rings'], values1=values1, values2=values2,  ylabel='count/Al', label1='CHA', label2='FER')

In [274]:
#str_vecs_no_1NN = random_config_mult(structure, str_clusters, 190, penalty=penalty, config_num=100, max_try = 100)

In [275]:
#str_vecs_no_rules = random_config_mult(structure, str_clusters, 190, penalty={}, config_num=100, max_try = 100)

In [276]:
#count_results_no_1NN = count_clusters_multi_configs(str_vecs_no_1NN, structure, str_clusters, counting_types=counting_types)

In [278]:
#count_results_no_rules = count_clusters_multi_configs(str_vecs_no_rules, structure, str_clusters, counting_types=counting_types)

In [ ]:
count_results_no_rules_values = [np.mean(count_results_no_rules[t])/190 for t in counting_types]
count_results_no_rules_errors = [np.std(count_results_no_rules[t])/190 for t in counting_types]
count_results_no_1NN_values = [np.mean(count_results_no_1NN[t])/190 for t in counting_types]
count_results_no_1NN_errors = [np.std(count_results_no_1NN[t])/190 for t in counting_types]

In [ ]:
plot_two_bar_charts(folder_path+'/rules vs no rules.png', counting_types, values1=count_results_no_rules_values, values2=count_results_no_1NN_values, errors1=count_results_no_rules_errors, errors2=count_results_no_1NN_errors, ylabel='count/Al', label1='no 1NN', label2='no rules')

#### count vs titrate

In [ ]:
counting_types = ['2-7', '2-11']

In [ ]:
#titrate_result = titrate_clusters_multi_configs(str_vecs_no_rules, structure, str_clusters, cluster_type_dict, counting_groups=[['2-7', '2-11']], titrate_num=100)

In [ ]:
#titrate_values = [np.mean(titrate_result[t]) for t in counting_types]
#titrate_errors = [np.std(titrate_result[t]) for t in counting_types]

In [ ]:
plot_two_bar_charts(folder_path+'/count vs titrate.png', counting_types, values1=count_results_no_rules_values, values2=titrate_values, errors1=count_results_no_rules_errors, errors=titrate_errors, ylabel='count', label1='count', label2='titrate')

#### composition

In [35]:
Si_Al_ratios = [2.5, 5, 7.5, 10, 15, 20, 25, 30, 35, 40]
Al_ratios = [1/(1+x) for x in Si_Al_ratios]
Al_nums = [277, 162, 114, 88, 60, 46, 37, 31, 27, 23]

In [36]:
values_cha = []
errors_cha = []
for i in range(10):
    filename = folder_path+'/jobs/titrate_results_{}.p'.format(i+1)
    titrate_result = pickle.load(open(filename, 'rb'))
    t = [(titrate_result['2-7'][j] + titrate_result['2-11'][j]) for j in range(len(titrate_result['2-7']))]
    values_cha.append(np.mean(t)/Al_nums[i])
    errors_cha.append(np.std(t)/Al_nums[i])

In [37]:
values_cha

[0.3276173285198556,
 0.23129629629629628,
 0.16631578947368422,
 0.12897727272727272,
 0.096,
 0.07217391304347825,
 0.06027027027027027,
 0.045806451612903226,
 0.03851851851851852,
 0.034782608695652174]

In [38]:
Si_Al_ratios = [2.5, 5, 7.5, 10, 15, 20, 25, 30, 35, 40]
Al_ratios = [1/(1+x) for x in Si_Al_ratios]
Al_nums = [164, 96, 67, 52, 36, 27, 22, 18, 16, 14]

In [ ]:
['2-6', '2-7', '2-9', '2-10', '2-20', '2-23']

In [41]:
values_fer = []
errors_fer = []
for i in range(10):
    filename = 'FER/jobs/titrate_results_{}.p'.format(i+1)
    titrate_result = pickle.load(open(filename, 'rb'))
    t = [(titrate_result['2-6'][i] + titrate_result['2-7'][i]+titrate_result['2-9'][i]+titrate_result['2-10'][i] + titrate_result['2-20'][i]+titrate_result['2-23'][i]) for i in range(len(titrate_result['2-7']))]
    values_fer.append(np.mean(t)/Al_nums[i])
    errors_fer.append(np.std(t)/Al_nums[i])

In [42]:
values_fer

[0.1775,
 0.10916666666666668,
 0.07850746268656716,
 0.06134615384615384,
 0.03944444444444444,
 0.030000000000000002,
 0.024090909090909093,
 0.020555555555555556,
 0.01375,
 0.016428571428571428]

#### jobs

In [197]:
import pickle

In [222]:
pickle.dump(lattice, open(folder_path+'/jobs/lattice.p','wb'))

In [223]:
pickle.dump(structure, open(folder_path+'/jobs/structure.p','wb'))

In [224]:
pickle.dump(lat_clusters, open(folder_path+'/jobs/lat_clusters.p','wb'))
pickle.dump(str_clusters, open(folder_path+'/jobs/str_clusters.p','wb'))

In [219]:
pickle.dump(penalty, open('penalty.p', 'wb'))

In [238]:
py_template = '''import pandas as pd
import numpy as np
import copy
from math import *
import os
import random
from collections import defaultdict
import csv
from functools import reduce
from copy import deepcopy
import pickle
from timeit import default_timer as timer
from matplotlib import pyplot as plt
from ase import Atoms
from ase.io import read
from ase.io import write
from ase.visualize import view
import itertools

from helper_functions import *

lattice = pickle.load(open('lattice.p','rb'))
structure = pickle.load(open('structure.p','rb'))
lat_clusters = pickle.load(open('lat_clusters.p','rb'))
str_clusters = pickle.load(open('str_clusters.p','rb'))
penalty = pickle.load(open('penalty.p','rb'))
Al_num = {0}
cluster_type_dict = prepare_cluster_type_dict(str_clusters)

str_vecs = random_config_mult(structure, str_clusters, Al_num, penalty, 100, max_try = 1000)
titrate_results = titrate_clusters_multi_configs(str_vecs, structure, str_clusters, cluster_type_dict, counting_groups=[['2-7', '2-11']], excluding_types=['3-6'], titrate_num=1)
pickle.dump(dict(titrate_results), open('titrate_results_{1}.p', 'wb'))
'''

In [226]:
Al_nums

[277, 162, 114, 88, 60, 46, 37, 31, 27, 23]

In [239]:
for i, ratio in enumerate(Si_Al_ratios):
    filename = folder_path+'/jobs/J{}.py'.format(i+1)
    if os.path.isfile(filename):
        os.remove(filename)
    with open(filename, 'a') as file:
        file.write(py_template.format(Al_nums[i], i+1))

In [228]:
qscript_template = '''#!/bin/bash
#$ -q *@@crc
#$ -pe smp 2
python J{}.py
'''

In [229]:
for i, ratio in enumerate(Si_Al_ratios):
    filename = folder_path+'/jobs/J{}.sh'.format(i+1)
    if os.path.isfile(filename):
        os.remove(filename)
    with open(filename, 'a') as file:
        file.write(qscript_template.format(i+1))

In [ ]:
titrate_results = []

In [ ]:
for i in range(1):
    filename = folder_path+'/jobs/titrate_results_{}.p'.format(i+1)
    titrate_result = pickle.load(open(filename, 'rb'))
    tirate_results.append(titrate_result)

In [232]:
titrate_results

[]

In [ ]:
titrate_results_1 = pickle